## Imports

In [3]:
import pandas as pd
import plotly.express as px
import kaleido

import os

## Data Paths

In [105]:
year = 2050

# data dir
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data', 'output_data')

# output figure dir
fig_dir = os.path.join(os.path.dirname(os.getcwd()), 'figures')

# path to suitability analysis df
suitability_data_path = os.path.join(data_dir, f'suitability_analysis_{year}.csv')

## Functions

## Collect Data

In [106]:
df = pd.read_csv(suitability_data_path)

solar = df[df.tech_name_simple == 'Solar PV']
solar_df = solar.melt(id_vars=['region_name', 'tech_name_simple'])

wind = df[df.tech_name_simple == 'Wind']

wind_df = wind.melt(id_vars=['region_name', 'tech_name_simple'])

bau_km_list = list(wind['bau_km'])

cg_km_list = list(wind['clean_grid_km'])



In [107]:
label_dict = {
    'baseline': 'Baseline Restrictions (BR)',
     'dac': 'BR + DACs',
     'farmland': 'BR + Important Farmland',
     'env_1km': 'BR + w/in 1km of Natural Areas',
     'env_5km': 'BR + w/in 5km of Natural Areas',
     'env_10km': 'BR + w/in 10km of Natural Areas',
     'dac_farm_env5km': 'BR + DACS, w/in 5km of Natural Area, and Important Farmland',
     'farm_env5km': 'BR + w/in 5km of Natural Area and Important Farmland',
     'farm_env10km': 'BR + w/in 10km of Natural Area and Important Farmland'}

wind_df = wind_df[~wind_df.variable.isin(['bau_km', 'clean_grid_km'])]
wind_df.variable = wind_df.variable.map(label_dict)

In [109]:


fig = px.bar(wind_df, x="variable", y="value", color="variable",
             facet_row="region_name", width=1200, height=2000,
            facet_row_spacing=0.03,
            color_discrete_sequence=px.colors.qualitative.Pastel)
state_list = list(wind_df.region_name.unique())

fig.update_yaxes(matches=None, title="square km", showgrid = True,  gridcolor = '#bdbdbd')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(showlegend=False, title='Wind Siting Availability under Various Exclusion Combinations',
                 uniformtext_minsize=18, uniformtext_mode='hide', plot_bgcolor = 'white', paper_bgcolor ='white')

for anno in fig['layout']['annotations']:
    anno['text']=''

start_y = 1.01
for state_label in state_label_list:
    fig.add_annotation(text=state_label,
                  xref="paper", yref="paper",
                  x=.5, y=start_y, showarrow=False, font=dict(size=16))
    start_y = start_y - .095 

row_no = 0
for i in cg_km_list:
    fig.add_hline(y=i, line_dash="dot", row=row_no,line_width=2,
          annotation_text=f'Clean Grid Required Land by {year}', 
          annotation_position="top left") 
    if row_no == 0:
        row_no = 10
    else:
        row_no-=1

fig.show()

In [111]:
solar_df = solar_df[~solar_df.variable.isin(['bau_km', 'clean_grid_km'])]
solar_df.variable = solar_df.variable.map(label_dict)

In [112]:


fig = px.bar(solar_df, x="variable", y="value", color="variable",
             facet_row="region_name", width=1200, height=2000,
            facet_row_spacing=0.03,
            color_discrete_sequence=px.colors.qualitative.Pastel)
state_list = list(solar_df.region_name.unique())

fig.update_yaxes(matches=None, title="square km", showgrid = True,  gridcolor = '#bdbdbd')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(showlegend=False, title='Solar Siting Availability under Various Exclusion Combinations',
                 uniformtext_minsize=18, uniformtext_mode='hide', plot_bgcolor = 'white', paper_bgcolor ='white')

for anno in fig['layout']['annotations']:
    anno['text']=''

start_y = 1.01
for state_label in state_label_list:
    fig.add_annotation(text=state_label,
                  xref="paper", yref="paper",
                  x=.5, y=start_y, showarrow=False, font=dict(size=16))
    start_y = start_y - .095 

row_no = 0
for i in cg_km_list:
    fig.add_hline(y=i, line_dash="dot", row=row_no,line_width=2,
          annotation_text=f'Clean Grid Required Land by {year}', 
          annotation_position="top left")
    if row_no == 0:
        row_no = 10
    else:
        row_no-=1

fig.show()

In [65]:
df = pd.read_csv(suitability_data_path)

solar = df[df.tech_name_simple == 'Solar PV']
solar_df = solar.melt(id_vars=['region_name', 'tech_name_simple'])

wind = df[df.tech_name_simple == 'Wind']

wind_df = wind.melt(id_vars=['region_name', 'tech_name_simple'])

bau_km_list = list(wind['bau_km'])

cg_km_list = list(wind['clean_grid_km'])



In [66]:
label_dict = {
    'baseline': 'Baseline Restrictions (BR)',
     'dac': 'BR + DACs',
     'farmland': 'BR + Important Farmland',
     'env_1km': 'BR + w/in 1km of Natural Areas',
     'env_5km': 'BR + w/in 5km of Natural Areas',
     'env_10km': 'BR + w/in 10km of Natural Areas',
     'dac_farm_env5km': 'BR + DACS, w/in 5km of Natural Area, and Important Farmland',
     'farm_env5km': 'BR + w/in 5km of Natural Area and Important Farmland',
     'farm_env10km': 'BR + w/in 10km of Natural Area and Important Farmland'}

wind_df = wind_df[~wind_df.variable.isin(['bau_km', 'clean_grid_km'])]
wind_df.variable = wind_df.variable.map(label_dict)

In [116]:


fig = px.bar(wind_df, x="variable", y="value", color="variable",
             facet_row="region_name", width=1200, height=2000,
            facet_row_spacing=0.03,
            color_discrete_sequence=px.colors.qualitative.Pastel)
state_list = list(wind_df.region_name.unique())

fig.update_yaxes(matches=None, title="square km", showgrid = True,  gridcolor = '#bdbdbd')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(showlegend=False, title='Wind Siting Availability under Various Exclusion Combinations',
                 uniformtext_minsize=18, uniformtext_mode='hide', plot_bgcolor = 'white', paper_bgcolor ='white')

for anno in fig['layout']['annotations']:
    anno['text']=''

start_y = 1.01
for state_label in state_label_list:
    fig.add_annotation(text=state_label,
                  xref="paper", yref="paper",
                  x=.5, y=start_y, showarrow=False, font=dict(size=16))
    start_y = start_y - .095 

row_no = 0
for i in cg_km_list:
    fig.add_hline(y=i, line_dash="dot", row=row_no,line_width=2,
          annotation_text='Clean Grid Required Land by 2035', 
          annotation_position="top left")
    if row_no == 0:
        row_no = 10
    else:
        row_no-=1

fig.show()

In [103]:
solar_df = solar_df[~solar_df.variable.isin(['bau_km', 'clean_grid_km'])]
solar_df.variable = solar_df.variable.map(label_dict)

In [104]:


fig = px.bar(solar_df, x="variable", y="value", color="variable",
             facet_row="region_name", width=1200, height=2000,
            facet_row_spacing=0.03,
            color_discrete_sequence=px.colors.qualitative.Pastel)
state_list = list(solar_df.region_name.unique())

fig.update_yaxes(matches=None, title="square km", showgrid = True,  gridcolor = '#bdbdbd')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(showlegend=False, title='Solar Siting Availability under Various Exclusion Combinations',
                 uniformtext_minsize=18, uniformtext_mode='hide', plot_bgcolor = 'white', paper_bgcolor ='white')

for anno in fig['layout']['annotations']:
    anno['text']=''

start_y = 1.01
for state_label in state_label_list:
    fig.add_annotation(text=state_label,
                  xref="paper", yref="paper",
                  x=.5, y=start_y, showarrow=False, font=dict(size=16))
    start_y = start_y - .095 

row_no = 0
for i in cg_km_list:
    fig.add_hline(y=i, line_dash="dot", row=row_no,line_width=2,
          annotation_text='Clean Grid Required Land by 2035', 
          annotation_position="top left")
    if row_no == 0:
        row_no = 10
    else:
        row_no-=1

fig.show()

In [32]:
print('solar')
fig = px.bar(solar_df, x="variable", y="value", color="variable",
             facet_row="region_name", width=1200, height=2000)
fig.update_yaxes(matches=None)
fig.show()

solar
